### Objective of the notbook.
1. Random Rotation Applied
2. Model is still Underfitting!

Output
*   Model Parameters: 8,144
*   Best Train Accuracy: 98.41%
*   Best Test Accuracy: 99.37%

### What can be improved?




In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary
import matplotlib.pyplot as plt

In [0]:
# Create the data reader and transformer
train_transform = transforms.Compose([transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,))])
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.1307,), (0.3081,))])


In [0]:
train = datasets.MNIST('./data',
                       train=True,
                       transform = train_transform,
                       download = True)
test = datasets.MNIST('./data',
                       train=False,
                       transform = test_transform,
                       download = True)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [0]:
# Set seed for all the environments
SEED = 1
torch.manual_seed(SEED)

CUDA = torch.cuda.is_available()
print("CUDA is available:",CUDA)
# If CUDA is available the set SEED for it
if CUDA:
    torch.cuda.manual_seed(SEED)


CUDA is available: True


In [0]:
# DATA LOADER
# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if CUDA else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train,**dataloader_args)
# test dataloader
test_loader = torch.utils.data.DataLoader(test,**dataloader_args)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # FIRST LAYER
        self.conv_block_1 =  nn.Sequential(nn.Conv2d(in_channels = 1,
                                                     out_channels = 8,
                                                     kernel_size=3, 
                                                     stride = 1,
                                                     padding=0,
                                                     bias=False
                                                     ),                          #output size = 26 Receptive field = 3
                                                     nn.BatchNorm2d(8),
                                                     nn.ReLU()
        )
        
        # CONVOLUTION BLOCK
        self.conv_block_2 = nn.Sequential(nn.Conv2d(in_channels=8,
                                                    out_channels=12,
                                                    kernel_size=3,
                                                    stride=1,
                                                    padding=0,
                                                    bias = False),                #Output Size = 24, Receptive Field = 5
                                          nn.BatchNorm2d(12),
                                          nn.ReLU(),
                                          nn.Dropout(0.08)
        ) 
        self.conv_block_3 = nn.Sequential(nn.Conv2d(in_channels=12,
                                                    out_channels=16,
                                                    kernel_size=3,
                                                    stride=1,
                                                    padding=0,
                                                    bias = False),                #Output size = 22 Receptive field = 7
                                          nn.BatchNorm2d(16),
                                          nn.ReLU(),
                                          nn.Dropout(0.10)
        )

        # TRANSITION BLOCK
        self.pool_1 = nn.Sequential(nn.MaxPool2d(kernel_size=2,
                                   stride=2,
                                   padding=0)                                    #Output size = 11 Receptive field = 8
         ) 
        self.conv_block_4 = nn.Sequential(nn.Conv2d(in_channels=16,
                                                    out_channels=8,
                                                    kernel_size=1,
                                                    stride=1,
                                                    padding=0,
                                                    bias = False),                #Output size = 11 Receptive field = 10
                                          nn.BatchNorm2d(8),
                                          nn.ReLU(),
                                          nn.Dropout(0.10)
        )

        # CONVOLUTION BLOCK
        self.conv_block_5 = nn.Sequential(nn.Conv2d(in_channels=8,
                                                    out_channels=12,
                                                    kernel_size=3,
                                                    stride=1,
                                                    padding=0,
                                                    bias = False),                #Output size = 9  Receptive field = 14
                                          nn.BatchNorm2d(12),
                                          nn.ReLU(),
                                          nn.Dropout(0.09)
        )
        self.conv_block_6 = nn.Sequential(nn.Conv2d(in_channels=12,
                                                    out_channels=16,
                                                    kernel_size=3,
                                                    stride=1,
                                                    padding=0,
                                                    bias = False),                #Output size = 7 Receptive field = 18
                                          nn.BatchNorm2d(16),
                                          nn.ReLU(),
                                          nn.Dropout(0.07)
        )
        self.conv_block_7 = nn.Sequential(nn.Conv2d(in_channels=16,
                                                    out_channels=16,
                                                    kernel_size=3,
                                                    stride=1,
                                                    padding=0,
                                                    bias = False),                #Output size = 5 Receptive field = 22
                                          nn.BatchNorm2d(16),
                                          nn.ReLU(),
                                          nn.Dropout(0.06)
        )
        
        # OUTPUT LAYER
        self.conv_block_8 = nn.Sequential(nn.Conv2d(in_channels=16,
                                                    out_channels=10,
                                                    kernel_size=1,
                                                    stride=1,
                                                    padding=0,
                                                    bias = False),                #Output size = 5 Receptive field = 24
                                          nn.BatchNorm2d(10),
                                          nn.ReLU(),
                                          nn.Dropout(0.06)
        )

        self.gap_layer = nn.Sequential(nn.AvgPool2d(5)) # output_size = 1
        self.conv_block_9 = nn.Sequential(nn.Conv2d(in_channels=10,
                                                    out_channels=10,
                                                    kernel_size=1,
                                                    stride=1,
                                                    padding=0,
                                                    bias = False)             #Output size = 5 Receptive field = 26
                                          
        )        
        #self.fc_layer = nn.Sequential(nn.Linear(10,10),nn.Dropout(0.05))

    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.conv_block_3(x)
        x = self.pool_1(x)
        x = self.conv_block_4(x)
        x = self.conv_block_5(x)
        x = self.conv_block_6(x)

        x = self.conv_block_7(x)
        x = self.conv_block_8(x)        
        x = self.gap_layer(x)
        #x = self.fc_layer(x)
        x = self.conv_block_9(x)
        x = x.view(-1, 10)
        final_x = F.log_softmax(x, dim=-1)
        return final_x

In [0]:
device = torch.device("cuda" if CUDA else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
       BatchNorm2d-2            [-1, 8, 26, 26]              16
              ReLU-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 12, 24, 24]             864
       BatchNorm2d-5           [-1, 12, 24, 24]              24
              ReLU-6           [-1, 12, 24, 24]               0
           Dropout-7           [-1, 12, 24, 24]               0
            Conv2d-8           [-1, 16, 22, 22]           1,728
       BatchNorm2d-9           [-1, 16, 22, 22]              32
             ReLU-10           [-1, 16, 22, 22]               0
          Dropout-11           [-1, 16, 22, 22]               0
        MaxPool2d-12           [-1, 16, 11, 11]               0
           Conv2d-13            [-1, 8, 11, 11]             128
      BatchNorm2d-14            [-

In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [0]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 0


Loss=0.17365765571594238 Batch_id=468 Accuracy=78.61: 100%|██████████| 469/469 [00:18<00:00, 24.94it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1472, Accuracy: 9639/10000 (96.39%)

EPOCH: 1


Loss=0.10504614561796188 Batch_id=468 Accuracy=96.60: 100%|██████████| 469/469 [00:18<00:00, 25.00it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0218, Accuracy: 9925/10000 (99.25%)

EPOCH: 13


Loss=0.016773829236626625 Batch_id=468 Accuracy=98.65: 100%|██████████| 469/469 [00:18<00:00, 25.28it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0205, Accuracy: 9937/10000 (99.37%)

EPOCH: 14


Loss=0.06628350168466568 Batch_id=468 Accuracy=98.69: 100%|██████████| 469/469 [00:18<00:00, 30.52it/s]



Test set: Average loss: 0.0242, Accuracy: 9921/10000 (99.21%)

